In [2]:
import numpy as np
from skylab.datasets import Dataset
#from skylab.datasets.NorthernTracks6yr import NorthernDataset
from datetime import datetime
import icecube
from icecube import astro
from astropy.time import Time
import datetime as datetime
import pandas as pd
import pickle
import h5py

In [3]:
path = '/data/ana/LE/NBI_nutau_appearance/current_pickle_files/current/merged_final/'

season_names = ['IC86-II', 'IC86-III', 'IC86-IV', 'IC86-V', 'IC86-VI']

outfile = '/data/user/apizzuto/Nova/GRECO_Skylab_Dataset/'

## Read in Random Forest parameters and Train on simulation

In [3]:
SimFiles = FileNames = ['/home/apizzuto/Nova/GRECO_nugen_numu_LE_with_PeglegCasc.h5', 
                        '/home/apizzuto/Nova/GRECO_nugen_numu_ME_with_PeglegCasc.h5', 
                        '/home/apizzuto/Nova/GRECO_genie_numu_with_PeglegCascade.h5']
NFiles = [4879, 644, 735]

def getTruth(neutrinoKey, weightKey):
    trueDict = {}
    for key in ['energy', 'zenith', 'azimuth', 'type', 
                'OneWeight', 'NEvents', 'MaxEnergyLog', 
                'MinEnergyLog', 'OneWeightbyNGen']:
        trueDict[key] = np.array([])
        
    for i in range(3):
        f = h5py.File(SimFiles[i], 'r')
        for item in ['energy', 'zenith', 'azimuth', 'type']:
            trueDict[item] = np.append(trueDict[item], f[neutrinoKey][item])
        for item in ['OneWeight', 'NEvents', 'MaxEnergyLog', 'MinEnergyLog']:
            trueDict[item] = np.append(trueDict[item], f[weightKey][item])
        trueDict['OneWeightbyNGen'] = np.append(trueDict['OneWeightbyNGen'], 
                                                f[weightKey]['OneWeight'] / (f[weightKey]['NEvents'] * NFiles[i]) )
    return trueDict

def getRecos(recoName, recoFits):
    recoDict = {}
    for key in ['energy', 'zenith', 'azimuth',
               'logl', 'rlogl']:
        recoDict[key] = np.array([])
        
    recoVars = ['energy', 'zenith', 'azimuth']
    recoParams = ['logl', 'rlogl']
    
    for i in range(3):
        f = h5py.File(SimFiles[i], 'r')
        for item in recoVars:
            recoDict[item] = np.append(recoDict[item], f[recoName][item])
        for item in recoParams:
            recoDict[item] = np.append(recoDict[item], f[recoFits][item])
            
    angularSeparation(recoDict, true)
    return recoDict

def angularSeparation(reco, truth):
    reco['angErr'] = np.arccos(np.sin(reco['zenith'])*np.sin(truth['zenith']) +
                        (np.cos(reco['zenith']) *np.cos(truth['zenith']) * np.cos(reco['azimuth'] - truth['azimuth'])) ) 

def maskAllRecos():
    mask = np.zeros_like(Pegleg_track['energy'])
    for reco in [Pegleg_track, Pegleg_cascade, Monopod, true]:
        mask += np.isnan(reco['energy'])
        mask += np.isnan(reco['zenith'])
        mask += np.isinf(reco['zenith'])
        mask += np.isinf(reco['energy']) #get rid of this if energy = 0 is helpful
    print np.unique(mask)
    print mask
    mask = map(bool, mask)
    mask = [not x for x in mask]
    for reco in [Pegleg_track, Pegleg_cascade, Monopod, true]:
        for key in reco.keys():
            reco[key] = reco[key][mask]
            
true = getTruth('MCNeutrino', 'I3MCWeightDict')
Pegleg_track = getRecos('Pegleg_Fit_MN_tol10Track', 'Pegleg_Fit_MN_tol10FitParams')
Pegleg_cascade = getRecos('Pegleg_Fit_MN_tol10HDCasc', 'Monopod_bestFitParams')
Monopod = getRecos('Monopod_best', 'Monopod_bestFitParams')
maskAllRecos()

[0.]
[0. 0. 0. ... 0. 0. 0.]


In [4]:
true

{'MaxEnergyLog': array([3.69897, 3.69897, 3.69897, ..., 3.     , 3.     , 3.     ]),
 'MinEnergyLog': array([2.        , 2.        , 2.        , ..., 1.90308999, 1.90308999,
        1.90308999]),
 'NEvents': array([50000., 50000., 50000., ..., 30700., 30700., 30700.]),
 'OneWeight': array([5.36788352e+06, 6.66118121e+09, 1.30386867e+07, ...,
        1.59149205e+05, 4.80256427e+05, 3.41449900e+05]),
 'OneWeightbyNGen': array([2.20040316e-02, 2.73055184e+01, 5.34481929e-02, ...,
        7.05307917e-03, 2.12837168e-02, 1.51321722e-02]),
 'azimuth': array([5.32977284, 1.6121022 , 0.57937745, ..., 0.75856769, 2.99386196,
        5.11008117]),
 'energy': array([104.42746359, 743.64107368, 111.27411143, ...,  88.80191194,
        128.05998938,  92.27591244]),
 'type': array([-14.,  14.,  14., ..., -14., -14.,  14.]),
 'zenith': array([2.19184033, 2.15648075, 2.24356555, ..., 2.67495411, 1.96634312,
        2.79588484])}

In [5]:
def predictSigma(exp):
    myForest = pickle.load(open('/data/user/apizzuto/Nova/RandomForests/FakeForestForEarlySensitivities.sav', 'rb'))
    
    df = {}
    df['Monopod_energy'] = FullExp['reco_cascade_energy']
    df['Pegleg_rlogl'] = FullExp['rLLH_Pegleg']
    df['Monopod_rlogl'] = FullExp['rLLH_Monopod']
    df['Pegleg_energy'] = FullExp['reco_track_energy']
    df['Pegleg_zenith'] = np.cos(FullExp['reco_zenith'])
    X_test = pd.DataFrame.from_dict(df)
    
    sigma = myForest.predict(X_test)
    return np.power(10, sigma)

In [57]:
for year in ['2012', '2013', '2014', '2015', '2016']:
    FullExp = np.load(path  + 'Level7_data_{}.pckl'.format(year))
    livetime = np.sum(FullExp['livetime']) / 86400. #livetime in days
    
    exp = {}
    

    exp['logE'] = exp['logE']
    exp['time'] = Time(FullExp['time']).mjd
    exp['run'] = FullExp['run']
    exp['event'] = FullExp['event_id']
    exp['ra'], exp['dec'] = astro.dir_to_equa(FullExp['reco_zenith'], FullExp['reco_azimuth'], exp['time']) 
    exp['azi'] = FullExp['reco_azimuth']
    exp['zenith'] = FullExp['reco_zenith']
    exp['angErr'] = predictSigma(exp)
    exp['logE'] = np.log10(FullExp['reco_energy']) ##FIGURE OUT MASK HERE NOW
    mask = exp['logE'] == np.isnan
    mask += exp['logE'] == -np.inf
    for key in exp.keys():
        exp[key] = exp[key[~mask]]
    dtype = []
    for key in sorted(exp.keys()):
        dtype.append((key, type(exp[key][0])))
    expVals = zip(*list(zip(*sorted(exp.items())))[1])
    exp = np.array(expVals, dtype = dtype)
    
    mc = {}
    
    mc['ra'], mc['dec'] = 
    mc['azi'] = 
    mc['zen'] = 
    mc['angErr'] = 
    mc['logE'] = 
    mc['trueE'] = 
    mc['trueRa'], mc['drueDec'] = 
    mc['ow'] = 
    mc['time'] = 
    dtype = []
    for key in sorted(mc.keys()):
        dtype.append((key, type(mc[key][0])))
    mcVals = zip(*list(zip(*sorted(mc.items())))[1])
    mc = np.array(mcVals, dtype = dtype)
    
    #np.save(outfile + season_names[i] + "_North_nugen", mc)
    #np.save(outfile + season_names[i] + "_North_data", exp)

In [58]:
exp

array([(0.40055002, 0.05924398, -0.74957049,  1930109, 13.32      , 4.90076149, 127951, 57528.87650724, 0.82168187),
       (0.36159041, 4.72469547, -0.24468621,  3142933,  6.18124319, 0.27053474, 127951, 57528.8818136 , 1.32766553),
       (0.23102447, 5.10804552,  0.07658806,  6884623, 27.10867273, 6.2733291 , 127951, 57528.89818219, 1.6490158 ),
       ...,
       (0.42196017, 5.69866536, -0.30759123, 70448516, 39.30893984, 4.6119834 , 129517, 57890.73722764, 1.26317025),
       (0.56076321, 2.53788971, -0.68481156, 71465419, 48.50086967, 1.51993582, 129517, 57890.74171706, 0.88588502),
       (0.81524434, 0.13940922, -1.18364048, 74774571, 45.17606974, 4.0053347 , 129517, 57890.7563128 , 0.38616482)],
      dtype=[('angErr', '<f8'), ('azi', '<f8'), ('dec', '<f8'), ('event', '<i8'), ('logE', '<f8'), ('ra', '<f8'), ('run', '<i8'), ('time', '<f8'), ('zenith', '<f8')])

In [5]:
FullExp

{'CE_contains_baddoms': array([False, False, False, ..., False, False, False]),
 'CalibrationErrataLength': array([0, 0, 0, ..., 0, 0, 0]),
 'GeV_per_channel': array([0.666     , 0.5619312 , 1.93633377, ..., 2.06889157, 2.42504348,
        3.22686212]),
 'StartingContainmentBool': array([ True,  True, False, ...,  True,  True, False]),
 'charge': array([20.37500007, 10.29999986, 13.97500007, ..., 21.1750001 ,
        20.99999979, 10.94999996]),
 'charge_fraction': array([0.1006135 , 0.18203883, 0.13059034, ..., 0.16174735, 0.09642858,
        0.1392694 ]),
 'charge_rms_normalized': array([0.23964104, 0.33260399, 0.29026113, ..., 0.26794751, 0.242968  ,
        0.29456479]),
 'deltaLLH': array([-1.80941791, -2.39931607, -2.70849005, ...,  0.29471975,
        -1.08133579, -5.78980379]),
 'dist_nearest_om': array([19.94972886, 21.18891896, 61.07311506, ..., 33.45075298,
        25.05376913, 57.03985873]),
 'dist_nearest_string': array([19.90999925, 20.99057596, 61.07287774, ..., 33.412633